# Embedding sample notebook

In [ ]:
from powerbi_client import Report

In [ ]:
import requests

# Embed config for sample report
res = requests.get('https://playgroundbe-bck-1.azurewebsites.net/Reports/SampleReport')

access_token = res.json()['EmbedToken']['Token']
embed_url = res.json()['EmbedUrl']

### Create an instance of Power BI Report

In [ ]:
report = Report(access_token, embed_url, token_type=1)

### Power BI event handlers

In [ ]:
def loaded_callback(event_details):
    print('loaded')

def rendered_callback(event_details):
    print('rendered')

def error_callback(event_details):
    print('error', event_details)

report.on('loaded', loaded_callback)
report.on('rendered', rendered_callback)
report.on('error', error_callback)

#### Load the report in the output cell

In [ ]:
# Render report
report

#### Change dimensions of embedded report

In [ ]:
# Change dimensions of embedded report
report.set_dimensions(300, 450)

### Export data from a visual of the embedded report

In [ ]:
def get_active_page(self):
    pages = self.get_pages()
    active_page = {}
    for page in pages:
        if page['isActive'] == True:
            active_page = page
    return active_page

In [ ]:
active_page = get_active_page(report)
active_page_name = active_page['name']

visuals = report.get_visuals(active_page_name)

visual_name = visuals[0]['name']

#### Export summarized data from a visual of the embedded report

In [ ]:
summarized_exported_data = report.extract_data(active_page_name, visual_name, rows = 5)

print(summarized_exported_data)

#### Export underlying data from a visual of the embedded report

In [ ]:
underlying_exported_data = report.extract_data(active_page_name, visual_name, rows = 5, underlying_data = True)

print(underlying_exported_data)

### Apply report filters

In [ ]:
filter = {
    '$schema': "http://powerbi.com/product/schema#basic",
    'target': {
        'table': "Geo",
        'column': "Region"
    },
    'operator': "In",
    'values': ["West"]
}
report.set_filters([filter])